In [ ]:
# switch to the project directory
%cd ..
# working directory should be ../FSE

In [ ]:
import sys
import os

module_path = os.path.abspath('src')

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pdi.constants import (
    NUM_WORKERS, )

from pdi.data.constants import N_COLUMNS
from pdi.data.types import Split
import wandb
from pdi.train import train


In [ ]:
import torch
import torch.nn as nn

torch.cuda.set_device(0)
device = torch.device("cuda")

In [ ]:
from math import sqrt
target_code = 321



def main(train_loader, val_loader, create_model_fun):
    wandb.init(allow_val_change=True)
    if "d_model" in wandb.config:
        wandb.config.start_lr = 1e-3 / sqrt(wandb.config.d_model)
    pos_weight = torch.tensor(1.0).float().to(device)
    wandb.log({"pos_weight": pos_weight.item()})
    model = create_model_fun()
    train(model, target_code, device, train_loader, val_loader, pos_weight)

Delete Sweep

In [ ]:
delete_sweep_configuration = {
    "method": "random",
    "metric": {
        "goal": "maximize",
        "name": "val_f1"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 10,
        "eta": 2,
    },
    "parameters": {
        "bs": {
            "value": 256
        },
        "max_epochs": {
            "value": 100
        },
        "start_lr": {
            "value": 5e-3
        },
        "patience": {
            "value": 10
        },
        "patience_threshold": {
            "value": 0.001
        },
        "h0": {
            "values": [16, 32, 64, 128, 256]
        },
        "h_layers": {
            "values": [1, 2, 3]
        },
        "h_scale": {
            "values": [0.125, 0.25, 0.5, 0.75]
        },
        "dropout": {
            "value": 0.1
        },
        "gamma": {
            "value": 0.9
        }
    },
}

In [ ]:
from pdi.models import NeuralNet


def create_simple_model():
    hidden_layers = [wandb.config.h0]
    for i in range(wandb.config.h_layers - 1):
        hidden_layers.append(int(wandb.config.h_scale * hidden_layers[-1]))

    return NeuralNet(
        [N_COLUMNS, *hidden_layers, 1],
        nn.ReLU,
        wandb.config.dropout,
    ).to(device)


In [ ]:
from pdi.data.preparation import DeletePreparation

sweep_id = wandb.sweep(sweep=delete_sweep_configuration, project="DeleteSweep")

bs = delete_sweep_configuration["parameters"]["bs"]["value"]
train_loader, val_loader = DeletePreparation().prepare_dataloaders(
       bs, NUM_WORKERS, [Split.TRAIN, Split.VAL])
wandb.agent(sweep_id, function=lambda: main(train_loader, val_loader, create_simple_model))

FSE Sweep

In [ ]:
fse_sweep_configuration = {
    "method": "random",
    "metric": {
        "goal": "minimize",
        "name": "val_loss"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 4,
        "eta": 2,
    },
    "parameters": {
        "bs": {
            "value": 512
        },
        "max_epochs": {
            "value": 40
        },
        "patience": {
            "value": 5
        },
        "patience_threshold": {
            "value": 0.001
        },
        "dropout": {
            "value": 0.1
        },
        "gamma": {
            "value": 0.9
        },
        "embed_hidden": {
            "values": [16, 32, 64, 128]
        },
        "d_model": {
            "values": [16, 32, 64]
        },
        "ff_hidden": {
            "values": [16, 32, 64, 128, 256]
        },
        "pool_hidden": {
            "values": [16, 32, 64, 128]
        },
        "num_heads": {
            "values": [1, 2, 4, 8]
        },
        "num_blocks": {
            "values": [1, 2, 4, 6]
        },
    },
}

In [ ]:
from pdi.models import AttentionModel


def create_attention_model():
    return AttentionModel(
        N_COLUMNS + 1,
        wandb.config.embed_hidden,
        wandb.config.d_model,
        wandb.config.ff_hidden,
        wandb.config.pool_hidden,
        wandb.config.num_heads,
        wandb.config.num_blocks,
        nn.ReLU,
        wandb.config.dropout,
    ).to(device)


In [ ]:
from pdi.data.preparation import FeatureSetPreparation

sweep_id = wandb.sweep(sweep=fse_sweep_configuration, project="FSESweep")

bs = fse_sweep_configuration["parameters"]["bs"]["value"]

train_loader, val_loader = FeatureSetPreparation().prepare_dataloaders(
        bs, NUM_WORKERS, [Split.TRAIN, Split.VAL])
wandb.agent(sweep_id, function=lambda: main(train_loader, val_loader, create_attention_model))